# Trend Analysis

# Database Creation

Using cleaned up CSVs:

Dish_clean.csv \
MenuPage_clean.csv \
MenuItem_clean.csv \
Menu_clean.csv

In [136]:
import sqlite3
import csv
import pandas as pd

First we make a connection to the *restaurant_menus.db* SQLite database file.\
If it does not exist, it will be automatically created.

In [137]:
conn = sqlite3.connect('restaurant_menus.db')

## 1. Inserting Dish data

Let's profile the Dish_clean.csv file by showing the head of the document and counts of the data as to know which fields should be not null and which ones can be null.

In [16]:
dish = pd.read_csv("Dish_clean.csv")
dish.head()

,id,name,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,1,"Consomme, Printaniere Royal",8,8,1897,1927,0.20,0.4
1,2,Chicken Gumbo,111,117,1895,1960,0.10,0.8
2,3,Tomato Aux Croutons,13,13,1893,1917,0.25,0.4
3,4,Onion Au Gratin,41,41,1900,1971,0.25,1.0
4,5,St Emilion,66,68,1881,1981,0.00,18.0


In [17]:
print(f"id count: {dish['id'].count()}")
print(f"name count: {dish['name'].count()}")
print(f"menus_appeared count: {dish['menus_appeared'].count()}")
print(f"times_appeared count: {dish['times_appeared'].count()}")
print(f"first_appeared count: {dish['first_appeared'].count()}")
print(f"last_appeared count: {dish['last_appeared'].count()}")
print(f"lowest_price count: {dish['lowest_price'].count()}")
print(f"highest_price count: {dish['highest_price'].count()}")

id count: 423397
name count: 423397
menus_appeared count: 423397
times_appeared count: 423397
first_appeared count: 423397
last_appeared count: 423397
lowest_price count: 394297
highest_price count: 394297


In [107]:
# USED FOR DIAGNOSTICS
# dish.loc[dish['id'] == '220797']

From this we see that only the **lowest_price** and **highest_price** columns can have null values.\
We will create the **dish** table with this in mind.

In [30]:
cursor = conn.cursor()
cursor.execute(
    '''
        CREATE TABLE IF NOT EXISTS dish (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            menus_appeared INTEGER NOT NULL,
            times_appeared INTEGER NOT NULL,
            first_appeared INTEGER NOT NULL,
            last_appeared INTEGER NOT NULL,
            lowest_price REAL,
            highest_price REAL
        )
    '''
)

conn.commit()

Next, the rows of the **Dish_clean.csv** file will be inserted into the **dish** table. \
The query to insert the CSV row data is specified first. \
We run the query for each row of data. 
We keep in mind that the *lowest_price* and *highest_price* columns could be null and account for that.

In [37]:
with open('Dish_clean.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    
    # Here we define the query to insert the dish items from Dish_clean.csv
    insert_query = '''
    INSERT INTO dish (id, name, menus_appeared, times_appeared, first_appeared, last_appeared, lowest_price, highest_price)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    '''
    
    # Insert each row from the Dish_clean.csv file.
    for row in csv_reader:
        # Account for the fact that lowest_price and highest_price can be null values.
        lowest_price = float(row['lowest_price']) if row['lowest_price'] else None
        highest_price = float(row['highest_price']) if row['highest_price'] else None
        try:
            cursor.execute(insert_query, (
                int(row['id']),
                row['name'],
                int(row['menus_appeared']),
                int(row['times_appeared']),
                int(row['first_appeared']),
                int(row['last_appeared']),
                lowest_price,
                highest_price
            ))
        except Exception as e:
            print(e)
            print(f"id: {row['id']}")

conn.commit()

In [41]:
# Check that all of the rows from Dish_clean.csv made it into the table.

cursor.execute('''
SELECT COUNT(*) FROM dish;
''')
rows = cursor.fetchall()
print(rows[0][0])

423397


## 2. Inserting Menu data
The rest of the data will be added in a similar fashion.

In [66]:
menu = pd.read_csv("Menu_clean.csv")
menu.head()

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
0,12463,NaN,HOTEL EASTMAN,BREAKFAST,COMMERCIAL,"HOT SPRINGS, AR",CARD; 4.75X7.5;,EASTER;,NaN,1900-2822,NaN,NaN,1900-04-15,Hotel Eastman,NaN,NaN,NaN,complete,2,67
1,12464,NaN,REPUBLICAN HOUSE,[DINNER],COMMERCIAL,"MILWAUKEE, [WI];",CARD; ILLUS; COL; 7.0X9.0;,EASTER;,WEDGEWOOD BLUE CARD; WHITE EMBOSSED GREEK KEY ...,1900-2825,NaN,NaN,1900-04-15,Republican House,NaN,NaN,NaN,under review,2,34
2,12465,NaN,NORDDEUTSCHER LLOYD BREMEN,FRUHSTUCK/BREAKFAST;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, STEAMSHIP A...",1900-2827,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,84
3,12466,NaN,NORDDEUTSCHER LLOYD BREMEN,LUNCH;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2828,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,63
4,12467,NaN,NORDDEUTSCHER LLOYD BREMEN,DINNER;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,FOLDER; ILLU; COL; 5.5X7.5;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2829,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,4,33


In [68]:
print(f"id count: {menu['id'].count()}")
print(f"name count: {menu['name'].count()}")
print(f"sponsor count: {menu['sponsor'].count()}")
print(f"event count: {menu['event'].count()}")
print(f"venue count: {menu['venue'].count()}")
print(f"place count: {menu['place'].count()}")
print(f"physical_description count: {menu['physical_description'].count()}")
print(f"occasion count: {menu['occasion'].count()}")
print(f"notes count: {menu['notes'].count()}")
print(f"call_number count: {menu['call_number'].count()}")
print(f"keywords count: {menu['keywords'].count()}")
print(f"language count: {menu['language'].count()}")
print(f"date count: {menu['date'].count()}")
print(f"location count: {menu['location'].count()}")
print(f"location_type count: {menu['location_type'].count()}")
print(f"currency count: {menu['currency'].count()}")
print(f"currency_symbol count: {menu['currency_symbol'].count()}")
print(f"status count: {menu['status'].count()}")
print(f"page_count count: {menu['page_count'].count()}")
print(f"dish_count count: {menu['dish_count'].count()}")

id count: 17545
name count: 3197
sponsor count: 15984
event count: 8154
venue count: 8119
place count: 8123
physical_description count: 14763
occasion count: 3791
notes count: 10612
call_number count: 15983
keywords count: 0
language count: 0
date count: 16959
location count: 17545
location_type count: 0
currency count: 6456
currency_symbol count: 6456
status count: 17545
page_count count: 17545
dish_count count: 17545


In [108]:
# USED FOR DIAGNOSTICS
# menu.loc[menu["id"] == 13112]

### Potential Issue
There are a total of 17545 menu records. \
Each menu has a non-null page count. \
There is a sum of 61134 total pages from this data. \
However, there are 66937 **MenuPage** records. \
Further analysis is required to see why that is happening.

In [70]:
print(f"page_count sum: {menu['page_count'].sum()}")

page_count sum: 61134


In [73]:
cursor = conn.cursor()
cursor.execute(
    '''
        CREATE TABLE IF NOT EXISTS menu (
            id INTEGER PRIMARY KEY,
            name TEXT,
            sponsor TEXT,
            event TEXT,
            venue TEXT,
            place TEXT,
            physical_description TEXT,
            occasion TEXT,
            notes TEXT,
            call_number TEXT,
            keywords TEXT,
            language TEXT,
            date TEXT,
            location TEXT NOT NULL,
            location_type TEXT,
            currency TEXT,
            currency_symbol TEXT,
            status TEXT NOT NULL,
            page_count INTEGER NOT NULL,
            dish_count INTEGER NOT NULL
        )
    '''
)

conn.commit()

In [76]:
with open('Menu_clean.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    
    # Here we define the query to insert the menus from Menu_clean.csv
    insert_query = '''
    INSERT INTO menu (id, name, sponsor, event, venue, place, physical_description, occasion,
                      notes, call_number, keywords, language, date, location, location_type,
                      currency, currency_symbol, status, page_count, dish_count)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    
    # Insert each row from the Dish_clean.csv file.
    for row in csv_reader:
        # Account for possible null values.
        name = row['name'] if row['name'] else None
        sponsor = row['sponsor'] if row['sponsor'] else None
        event = row['event'] if row['event'] else None
        venue = row['venue'] if row['venue'] else None
        place = row['place'] if row['place'] else None
        physical_description = row['physical_description'] if row['physical_description'] else None
        occasion = row['occasion'] if row['occasion'] else None
        notes = row['notes'] if row['notes'] else None
        call_number = row['call_number'] if row['call_number'] else None
        keywords = row['keywords'] if row['keywords'] else None
        language = row['language'] if row['language'] else None
        date = row['date'] if row['date'] else None
        location_type = row['location_type'] if row['location_type'] else None
        currency = row['currency'] if row['currency'] else None
        currency_symbol = row['currency_symbol'] if row['currency_symbol'] else None
        
        try:
            cursor.execute(insert_query, (
                int(row['id']), name, sponsor, event, venue, place, physical_description, occasion, notes,
                call_number, keywords, language, date, row['location'], location_type, currency,
                currency_symbol, row['status'], int(row['page_count']), int(row['dish_count'])
            ))
        except Exception as e:
            print(e)
            print(f"id: {row['id']}")

conn.commit()

## 3. Inserting MenuPage data
The rest of the data will be added in a similar fashion.

In [77]:
menu_page = pd.read_csv("MenuPage_clean.csv")
menu_page.head()

,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99


In [78]:
print(f"id count: {menu_page['id'].count()}")
print(f"menu_id count: {menu_page['menu_id'].count()}")
print(f"page_number count: {menu_page['page_number'].count()}")
print(f"image_id count: {menu_page['image_id'].count()}")
print(f"full_height count: {menu_page['full_height'].count()}")
print(f"full_width count: {menu_page['full_width'].count()}")
print(f"uuid count: {menu_page['uuid'].count()}")

id count: 66937
menu_id count: 66937
page_number count: 65735
image_id count: 66937
full_height count: 66608
full_width count: 66608
uuid count: 66937


In [80]:
cursor.execute('''
DROP TABLE IF EXISTS menu_page;
''')
conn.commit()

In [82]:
cursor = conn.cursor()
cursor.execute(
    '''
        CREATE TABLE IF NOT EXISTS menu_page (
            id INTEGER PRIMARY KEY,
            menu_id INTEGER NOT NULL,
            page_number INTEGER,
            image_id TEXT NOT NULL,
            full_height REAL,
            full_width REAL,
            uuid TEXT NOT NULL,
            FOREIGN KEY(menu_id) REFERENCES menu(id)
        )
    '''
)

conn.commit()

In [83]:
with open('MenuPage_clean.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    
    # Here we define the query to insert the menu page items from MenuPage_clean.csv
    insert_query = '''
    INSERT INTO menu_page (id, menu_id, page_number, image_id, full_height, full_width, uuid)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    '''
    
    # Insert each row from the MenuPage_clean.csv file.
    for row in csv_reader:
        # Account for the fact that page_number, full_height and full_width can be null values.
        page_number = int(row['page_number']) if row['page_number'] else None
        full_height = float(row['full_height']) if row['full_height'] else None
        full_width = float(row['full_width']) if row['full_width'] else None
        try:
            cursor.execute(insert_query, (
                int(row['id']),
                int(row['menu_id']),
                page_number,
                row['image_id'],
                full_height,
                full_width,
                row['uuid']
            ))
        except Exception as e:
            print(e)
            print(f"id: {row['id']}")

# Commit the changes and close the connection
conn.commit()

In [84]:
# Check that all of the rows from MenuPage_clean.csv made it into the table.

cursor.execute('''
SELECT COUNT(*) FROM menu_page;
''')
rows = cursor.fetchall()
print(rows[0][0])

66937


## 4. Inserting MenuItem data
The rest of the data will be added in a similar fashion.

In [85]:
menu_item = pd.read_csv("MenuItem_clean.csv")
menu_item.head()

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,1,1389,0.4,NaN,1.0,2011-03-28 15:00:44 UTC,2011-04-19 04:33:15 UTC,0.111429,0.254735
1,2,1389,0.6,NaN,2.0,2011-03-28 15:01:13 UTC,2011-04-19 15:00:54 UTC,0.438571,0.254735
2,3,1389,0.4,NaN,3.0,2011-03-28 15:01:40 UTC,2011-04-19 19:10:05 UTC,0.140000,0.261922
3,4,1389,0.5,NaN,4.0,2011-03-28 15:01:51 UTC,2011-04-19 19:07:01 UTC,0.377143,0.262720
4,5,3079,0.5,1.0,5.0,2011-03-28 15:21:26 UTC,2011-04-13 15:25:27 UTC,0.105714,0.313178


In [86]:
print(f"id count: {menu_item['id'].count()}")
print(f"menu_page_id count: {menu_item['menu_page_id'].count()}")
print(f"price count: {menu_item['price'].count()}")
print(f"high_price count: {menu_item['high_price'].count()}")
print(f"dish_id count: {menu_item['dish_id'].count()}")
print(f"created_at count: {menu_item['created_at'].count()}")
print(f"updated_at count: {menu_item['updated_at'].count()}")
print(f"xpos count: {menu_item['xpos'].count()}")
print(f"ypos count: {menu_item['ypos'].count()}")

id count: 1332726
menu_page_id count: 1332726
price count: 886810
high_price count: 91905
dish_id count: 1332485
created_at count: 1332726
updated_at count: 1332726
xpos count: 1332726
ypos count: 1332726


In [109]:
# USED FOR DIAGNOSTICS
# menu_item.loc[menu_item["id"] == 619133]

### Potential Issue
Each menu item should refer to a single dish from the **dish** table, and should not be null. \
However, there are 241 null values in **dish_id** column rows. (1332726 - 1332485) \
While it may not be possible to find out which specific dishes these items refer to, it can still be beneficial
to keep them in the database \
table in order to answer other questions such as, what were the average menu item prices per restaurant regardless of what the dish is.

In [96]:
cursor.execute('''
DROP TABLE IF EXISTS menu_item;
''')
conn.commit()

In [97]:
cursor.execute(
    '''
        CREATE TABLE IF NOT EXISTS menu_item (
            id INTEGER PRIMARY KEY,
            menu_page_id INTEGER NOT NULL,
            price REAL,
            high_price REAL,
            dish_id INTEGER,
            created_at TEXT NOT NULL,
            updated_at TEXT NOT NULL,
            xpos REAL NOT NULL,
            ypos REAL NOT NULL,
            FOREIGN KEY(menu_page_id) REFERENCES menu_page(id),
            FOREIGN KEY(dish_id) REFERENCES dish(id)
        )
    '''
)

conn.commit()

In [98]:
with open('MenuItem_clean.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    
    # Here we define the query to insert the menu items from MenuItem_clean.csv
    insert_query = '''
    INSERT INTO menu_item (id, menu_page_id, price, high_price, dish_id, created_at, updated_at, xpos, ypos)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    
    # Insert each row from the MenuItem_clean.csv file.
    for row in csv_reader:
        # Account for the fact that page_number, full_height and full_width can be null values.
        price = float(row['price']) if row['price'] else None
        high_price = float(row['high_price']) if row['high_price'] else None
        dish_id = int(row['dish_id']) if row['dish_id'] else None
        try:
            cursor.execute(insert_query, (
                int(row['id']), int(row['menu_page_id']), price, high_price, dish_id,
                row['created_at'], row['updated_at'], float(row['xpos']), float(row['ypos'])
            ))
        except Exception as e:
            print(e)
            print(f"id: {row['id']}")

# Commit the changes and close the connection
conn.commit()

## 5. Getting the Top 10 Menu Items on an Annual Basis
In order to help us find our answer for Use Case 1, we need to query the data that is now stored inside the SQLite database. \
The query will provide us the top 10 most occuring menu items for each year, ranked. \
This can be further used to create a graph.

In [138]:
cursor = conn.cursor()
cursor.execute(
    '''
        SELECT *
        FROM
        (
                SELECT
                    CAST(substr(menu.date, 1, 4) as INTEGER) AS Year,
                    dish.name AS 'DishName',
                    COUNT(dish.name) AS Occurences,
                    ROW_NUMBER () OVER (
                        PARTITION BY CAST(substr(menu.date, 1, 4) as INTEGER)
                        ORDER BY COUNT(dish.name) DESC, dish.name
                    ) Rank
                FROM
                        menu_item
                        LEFT OUTER JOIN dish ON menu_item.dish_id = dish.id
                        LEFT OUTER JOIN menu_page ON menu_item.menu_page_id = menu_page.id
                        LEFT OUTER JOIN menu ON menu_page.menu_id = menu.id
                WHERE 
                        menu_item.dish_id NOT NULL AND
                        menu.date NOT NULL AND
                        menu.date NOT LIKE '0%' AND
                        menu.date NOT LIKE '10%' AND
                        dish.id NOT NULL AND
                        (dish.name NOT LIKE '"' AND dish.name NOT LIKE '" "%')
                GROUP BY Year, dish.name
        ) t

        WHERE Rank <= 10
    '''
)

fields = ['Year', 'DishName', 'Occurences', 'Rank']

rows = cursor.fetchall()

filename = "menu_item_historical_frequencies.csv"

with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    
    for row in rows:
        csvwriter.writerow(list(row))